In [477]:
from time import process_time

import torch
from rl4co.envs.scheduling.djssp.env import DJSSPEnv
from rl4co.models import L2DPolicy, L2DModel
from rl4co.utils import RL4COTrainer
import gc
from rl4co.envs import JSSPEnv
from rl4co.models.zoo.l2d.model import L2DPPOModel
from rl4co.models.zoo.l2d.policy import L2DPolicy4PPO
from torch.utils.data import DataLoader
import json
import os
%load_ext autoreload
%autoreload 2
generator_params = {
    "num_jobs" : 6 ,
    "num_machines":  6 ,
    "min_processing_time": 1 ,
    "max_processing_time": 99 ,
    "mtbf" : 17 ,
    "mttr" : 4
}
env = DJSSPEnv(generator_params=generator_params,
               _torchrl_mode=True,
            stepwise_reward=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [478]:
td = env.reset(batch_size=[1]) 

In [479]:
td["start_op_per_job"].size(1)

6

In [480]:

import collections
from ortools.sat.python import cp_model

# job arrival times
arrival_times = td["job_arrival_times"][0]

# [[operation_id, machine_id, proc_time]]
fin = env.get_op_ma_proctime(td)

# create the jobs_data array
# [ job [machine_no , proc_time]]
jobs_data = []
num_jobs = td["start_op_per_job"].size(1)

# add empty array for each job in jobs_data
for i in range(num_jobs):
    jobs_data.append([])

for x in range ((num_jobs*num_jobs+1)-1):
    # her job esit sayida operation'a sahip
    job_no = x // num_jobs
    # task (machine_id , processing_time)
    task = (fin[x][1] , fin[x][2])
    jobs_data[job_no].append(task)


#horizon = torch.sum(td["proc_times"][0]).item()
horizon = sum(op[2] for op in fin)



# declare the model
model = cp_model.CpModel()


                            # define the variables
# create a named tuple to store information about created varibles
task_type = collections.namedtuple("task_type" , "start end interval")

# create a named tuple to manipulate solution information
assigned_task_type = collections.namedtuple("assigned_task_type" , "start job index duration")

# create job intervals and add to the corresponding MACHINE LIST
all_task = {}
machine_to_intervals = collections.defaultdict(list)

all_machines = range(env.num_mas)

# OR-Tools does not support float type numbers therefore we reound all of them

for job_id , job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        # get the machine id and the duration from the task
        machine , duration = task
        # AttributeError: 'float' object has no attribute 'get_integer_var_value_map'
        duration = int(duration) +1
        suffix = f"_{job_id}_{task_id}"
        #Create an integer variable with domain [lb, ub]. [0,horizon] "start time of the specific task"
        # TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
        # 1. ortools.util.python.sorted_interval_list.Domain(arg0: int, arg1: int)
        # Invoked with: 0, 1929.2211356163025
        horizon = int(horizon) +1
        # start time of the task is created here
        job_arrival_time= int(arrival_times[job_id].item())+1

        # start_var = model.new_int_var(0 , horizon , "start"+ suffix)
        start_var = model.new_int_var(job_arrival_time , horizon , "start"+ suffix)

        # create ending time of the specific task using constraint programming model (end_0_0...)
        # final/end time of the specific task is created here
        # end_var = model.new_int_var(0, horizon , "end"+ suffix)
        end_var = model.new_int_var(job_arrival_time, horizon , "end"+ suffix)
        #  create interval variable from start_var duration end_var (interval_0_2.....)
        interval_var = model.new_interval_var(
            start_var, duration , end_var , "interval" + suffix
        )

        #TODO: machine breakdowns
        # create updated_end variable to use if there is a machine breakdown
        updated_end = model.new_int_var(0, horizon, "adjusted_end" + suffix)
        # extract the breakdowns of the machine
        breakdowns = td["machine_breakdowns"][0,machine]
        # extract the breakdown occurrence times
        occurrences = breakdowns[::2]  # to get the only even indices (occurrence times)
        # extract the breakdown durations
        durations = breakdowns[1::2]  # to get the only odd indices (durations)

        # we need to add constraint on breakdown
        for breakdwn_no in range(occurrences.size(0)-1):
            # occurence time
            occ_time = int(occurrences[breakdwn_no].item())
            # duration
            duration = int(durations[breakdwn_no].item())+1
            # i have padded the tensor with values 0 if there is no breakdown, therefore ignore these
            if occ_time==0 and duration==0:
                continue
            #todo: burayi comment out yaptim
            # check if breakdown happens when an operation is being processed on machine
            # create condition variable
            breakdown_condition = model.new_bool_var(f"breakdown_{suffix}_{breakdwn_no}")
            # same logic as in makestep
            # operation starts before breakdown ends
            model.add( start_var < (occ_time + duration)).only_enforce_if(breakdown_condition)
            # operation ends after breakdowns starts
            model.add( (start_var + duration)>occ_time ).only_enforce_if(breakdown_condition)

            # if there is breakdown add duration to the updated end
            model.add(updated_end == end_var + duration).only_enforce_if(breakdown_condition)

        # if there is no breakdown during operaion then updated_end  = end_var
        no_breakdown = model.new_bool_var(f"no_breakdown_{suffix}")
        model.add(updated_end == end_var).only_enforce_if(no_breakdown)
        model.add_bool_or([no_breakdown] + [model.new_bool_var(f"breakdown_{suffix}_{idx}") for idx in range(occurrences.size(0))])

        # add all the task's with start,interval,end informations in all_task dict
        all_task [ job_id , task_id ] = task_type(
            start = start_var,
            end = end_var,
            interval = interval_var
        )
        #            end = updated_end,

        # add at each machine index the operations/tasks interval where it containes start, end, duration
        machine_to_intervals[machine].append(interval_var)



C:\Users\soner\Desktop\rl4co\rl4co\envs\scheduling\djssp\env.py:848: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(td["proc_times"][0])


In [481]:
# DEFINE THE CONSTRAINTS

# create and add disjunctive constraints
for machine in all_machines:
    # use add_no_overlap method to create no overlap constrains
    # to prevent tasks for the same machine from overlapping time
    model.add_no_overlap(machine_to_intervals[machine])


# precedences inside a job
for job_id , job in enumerate(jobs_data):
    for task_id in range(len(job) -1 ):
        model.add(
            all_task[ job_id , task_id +1 ].start >= all_task[ job_id , task_id].end
        )


## Define the objective
    - This code creates an objective variable and constrains it to be the max of the end of all jobs.

In [482]:
# Makespan objective
#create a new integer variable for the makespan (obj_var is the makespan)
obj_var = model.new_int_var(0 , horizon , "makespan") #makespan(0..21)

# add constraint to make sthe makespan to the last task of all jobs
# obj_var(makespan) is equal to latest end time of all task
# obj_var == max (ebd times of all tasks)
model.add_max_equality(
    obj_var , [all_task[ job_id , len(job) -1 ].end for job_id , job in enumerate(jobs_data)],
                       )

# set objective to minimize the makespan
model.minimize(obj_var)

# Invoke the solver


In [483]:
solver = cp_model.CpSolver()
status = solver.solve(model)

In [484]:
# Check if solution was found
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solution:")
    # Create one list of assigned tasks per machine
    # keys = mcahine IDs (0,1,2.---)
    # values = list of tasks assigned to each machine
    assigned_jobs = collections.defaultdict(list)
    # iterate through all jobs
    for job_id, job in enumerate(jobs_data):
        # job_id = ID of the job 0,1,2..
        # job = list of tasks for that job [(0, 3), (1, 2), (2, 2)]

        # iterate over tasks for that job
        for task_id, task in enumerate(job):
            #task_id = index of the task in job  0,1,2..
            # task tuple (0, 3) : (machine_id, proc_time)

            machine = task[0]
            # add tasks details to the machines list
            assigned_jobs[machine].append(
                assigned_task_type(
                    start=solver.value(all_task[job_id, task_id].start),
                    job=job_id,
                    index=task_id,
                    duration=task[1],
                )
            )
    # Create per machine output lines.
    output = ""
    for machine in all_machines:
        # Sort by starting time.
        assigned_jobs[machine].sort()
        sol_line_tasks = "Machine " + str(machine) + ": "
        sol_line = "              "

        for assigned_task in assigned_jobs[machine]:
            name = f"job_{assigned_task.job}_task_{assigned_task.index}       "
            # add spaces to output to align columns.
            sol_line_tasks += f"{name:15}"
            # TODO: !!!!!!!!
            start = assigned_task.start
            duration = assigned_task.duration
            sol_tmp = f"[{start},{start + duration}]"
            # add spaces to output to align columns.
            sol_line += f"{sol_tmp:15}"

        sol_line += "\n"
        sol_line_tasks += "\n"
        output += sol_line_tasks
        output += sol_line

    # Finally print the solution found.
    print(f"Optimal Schedule Length: {solver.objective_value}")
    print(output)
else:
    print("No solution found.")

Solution:
Optimal Schedule Length: 818.0
Machine 0: job_1_task_3       job_2_task_2       job_0_task_4       job_5_task_4       job_3_task_5       job_4_task_5       
              [319,352.6509323120117][353,428.61541748046875][429,430.0350989103317][518,532.4734506607056][623,716.3310546875][731,817.1169738769531]
Machine 1: job_2_task_0       job_0_task_3       job_3_task_1       job_1_task_4       job_5_task_3       job_4_task_4       
              [189,259.2629623413086][260,323.9852867126465][348,365.30669593811035][366,421.922119140625][509,517.9966859817505][659,730.3690643310547]
Machine 2: job_0_task_2       job_1_task_1       job_4_task_0       job_5_task_5       job_3_task_4       job_2_task_5       
              [122,222.28093719482422][223,285.85128021240234][319,388.4849090576172][533,562.8181495666504][593,622.12917137146][680,707.0809097290039]
Machine 3: job_0_task_0       job_1_task_2       job_4_task_1       job_5_task_2       job_3_task_3       job_2_task_4      

- machine breakdown'lari bir sekilde eklemek gerekiyor

In [485]:
td["job_arrival_times"]

tensor([[ 47.4129,  56.5622, 188.9430, 251.7938, 318.0958, 334.0412]])

In [486]:
td["proc_times"][0]

tensor([[  0.0000,   0.0000,   0.0000,   0.0000,   1.0351,   0.0000,   0.0000,
           0.0000,   0.0000,  33.6509,   0.0000,   0.0000,   0.0000,   0.0000,
          75.6154,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,  93.3311,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,  86.1170,   0.0000,   0.0000,   0.0000,   0.0000,  14.4735,
           0.0000],
        [  0.0000,   0.0000,   0.0000,  63.9853,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,  55.9221,   0.0000,  70.2630,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,  17.3067,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          71.3691,   0.0000,   0.0000,   0.0000,   0.0000,   8.9967,   0.0000,
           0.0000],
        [  0.0000,   0.0000, 100.2809,   0.0000,   0.0000,   0.0000,   0.0000,
          62.8513,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0

In [487]:
252 + 9.167562

261.167562

In [488]:
print(td["machine_breakdowns"][0])

tensor([[4.6538e+01, 5.0296e+00, 6.3100e+01, 1.0097e+01, 1.3300e+02, 6.9801e+00,
         2.7412e+02, 1.0043e+00, 5.5139e+02, 9.9975e+00, 1.1137e+03, 7.5229e-02,
         2.2202e+03, 8.8703e-01, 4.4504e+03, 3.7722e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [8.9034e+00, 3.2588e+00, 1.3309e+01, 3.5548e+00, 3.1885e+01, 3.2662e+00,
         7.1873e+01, 2.8204e+00, 1.4057e+02, 3.5399e-01, 2.8055e+02, 9.2605e+00,
         5.7872e+02, 3.7138e-01, 1.1748e+03, 2.7541e+00, 2.3613e+03, 7.6855e+00,
         4.7021e+03, 1.2679e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.8262e+01, 4.8151e-01, 6.3443e+01, 6.5475e+00, 1.5014e+02, 2.4163e+00,
         2.5974e+

In [489]:
td["machine_breakdowns"][0]

tensor([[4.6538e+01, 5.0296e+00, 6.3100e+01, 1.0097e+01, 1.3300e+02, 6.9801e+00,
         2.7412e+02, 1.0043e+00, 5.5139e+02, 9.9975e+00, 1.1137e+03, 7.5229e-02,
         2.2202e+03, 8.8703e-01, 4.4504e+03, 3.7722e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [8.9034e+00, 3.2588e+00, 1.3309e+01, 3.5548e+00, 3.1885e+01, 3.2662e+00,
         7.1873e+01, 2.8204e+00, 1.4057e+02, 3.5399e-01, 2.8055e+02, 9.2605e+00,
         5.7872e+02, 3.7138e-01, 1.1748e+03, 2.7541e+00, 2.3613e+03, 7.6855e+00,
         4.7021e+03, 1.2679e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.8262e+01, 4.8151e-01, 6.3443e+01, 6.5475e+00, 1.5014e+02, 2.4163e+00,
         2.5974e+

In [490]:
  # Retrieve machine breakdown times and durations
machine = 0
breakdowns = td["machine_breakdowns"][0,machine]
breakdowns

tensor([4.6538e+01, 5.0296e+00, 6.3100e+01, 1.0097e+01, 1.3300e+02, 6.9801e+00,
        2.7412e+02, 1.0043e+00, 5.5139e+02, 9.9975e+00, 1.1137e+03, 7.5229e-02,
        2.2202e+03, 8.8703e-01, 4.4504e+03, 3.7722e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00])

In [491]:
breakdown_occurrences = breakdowns[::2]  # to get the only even indices (occurence times)
breakdown_occurrences

tensor([  46.5375,   63.1001,  132.9976,  274.1193,  551.3894, 1113.7128,
        2220.1628, 4450.4224,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000])

In [492]:
breakdown_durations = breakdowns[1::2]  # to get the only odd indices (durations)
breakdown_durations

tensor([ 5.0296, 10.0965,  6.9801,  1.0043,  9.9975,  0.0752,  0.8870,  3.7722,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])